In [1]:
!free -h


               total        used        free      shared  buff/cache   available
Mem:            12Gi       6.8Gi       824Mi       1.0Mi       5.1Gi       5.5Gi
Swap:             0B          0B          0B


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define dataset paths (FIXED PATH)
data_dir = "/content/drive/MyDrive/Video-dataset"
categories = ["consent", "nonconsent"]
img_size = (64, 64)  # Resize frame to 64x64 pixels

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
base_model.trainable = False  # Freeze base model

def load_videos():
    data, labels = [], []
    for label, category in enumerate(categories):
        category_path = os.path.join(data_dir, category)
        if not os.path.exists(category_path):  # Check if directory exists
            print(f"Warning: {category_path} does not exist!")
            continue

        for video_name in os.listdir(category_path):
            video_path = os.path.join(category_path, video_name)
            cap = cv2.VideoCapture(video_path)
            frames = []

            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, img_size)
                frames.append(frame)

            cap.release()
            if frames:
                data.append(frames)
                labels.append(label)

    if not data:
        raise ValueError("No videos found in dataset!")

    max_frames = max(len(vid) for vid in data)
    data_padded = [vid + [np.zeros((64, 64, 3))] * (max_frames - len(vid)) for vid in data]
    return np.array(data_padded) / 255.0, np.array(labels)

# Load data
X, y = load_videos()
X = np.array([np.array(vid) for vid in X])  # Convert to proper numpy array
y = keras.utils.to_categorical(y, num_classes=2)

# Split into train & test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN-LSTM Model
model = keras.Sequential([
    layers.TimeDistributed(base_model, input_shape=(None, 64, 64, 3)),
    layers.TimeDistributed(layers.Flatten()),  # Convert 3D feature maps to sequence for LSTM
    layers.LSTM(64, return_sequences=False),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

# Compile the model with learning rate scheduler
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Create model checkpoint directory
checkpoint_dir = "/content/drive/MyDrive/model_checkpoints"  # Save checkpoints in Drive
os.makedirs(checkpoint_dir, exist_ok=True)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model_checkpoint = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}.keras"),
    save_freq='epoch',
    period=2  # Save every 2 epochs
)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=4, callbacks=[early_stop, lr_scheduler, model_checkpoint])

# Save final model
model_save_path = "/content/drive/MyDrive/video_classifier.keras"  # Save model in Drive
model.save(model_save_path)
print(f"Model trained and saved as {model_save_path}")


<ipython-input-3-8772979fe67b>:16: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
